In [1]:
import pandas as pd
import numpy as np
import json
from scipy import stats

In [2]:
datasets = [{"filepath": r"../Resources_cleaned/winemag_full.json",
           "name": "full"},
            {"filepath": r"../Resources_cleaned/winemag_23000.json",
           "name": "23k"},
          {"filepath": r"../Resources_cleaned/winemag_5000.json",
          "name": "5k"},
           ]


In [22]:
df_list = []
for dd in datasets:
    df = pd.read_json(dd["filepath"])
    nb_rows = df.shape[0]
    df_temp = df.groupby("country")['points'].count()/nb_rows
    df_temp = df_temp.rename(dd["name"])
    df_list.append(df_temp)
#     print(df.shape[0])
#     # groupby country and extract the nb of row per country
#     print(df["country"].unique())
    


In [34]:
df_list
merged_dt = pd.concat(df_list, axis=1, sort=False)
# pd.concat([s1, s2], axis=1)
merged_dt = merged_dt.fillna(0)
merged_dt

,full,23k,5k
Argentina,0.035009,0.046851,0.046875
Australia,0.021382,0.064796,0.064844
Chile,0.041160,0.050364,0.050391
China,0.000009,0.000000,0.000000
France,0.165685,0.216861,0.216797
Italy,0.157650,0.253724,0.253711
South Africa,0.012052,0.052353,0.052344
Spain,0.061265,0.195912,0.195898
US,0.505788,0.119138,0.119141


In [35]:
# Sample the database based on the wine production on 2016
# loading the data
prod_file = r"../Resources_raw/wine_production_by_country_2016.csv"
df_prod = pd.read_csv(prod_file,
                      encoding="utf-8",
                      header=None,
                     names=["Country","Year","production"])
df_prod = df_prod.drop(["Year"], axis=1)
# calculating the total production this year
total_production = df_prod["production"].sum()
total_production
# Normalize the production
df_prod["production"] = df_prod["production"]/total_production
# Replace United State by US to match the main dataset labels
df_prod['Country'] = df_prod['Country'].str.replace("United States", "US")

# We will select only the 10 bigest countries
df_prod2 = df_prod[0:9]
# China has only one wine reviewed in the dataset so we just remove it
# df_prod2 = df_prod2.drop(5) 
# df_prod2

df_prod2 = df_prod2.set_index('Country')
df_prod2

,production
Country,
Italy,0.199843
France,0.170789
Spain,0.154299
US,0.093836
Australia,0.051040
China,0.044759
South Africa,0.041225
Chile,0.039654
Argentina,0.036906


In [36]:
merged_dt2 = pd.concat([df_prod2["production"],merged_dt], axis=1, sort=False)
merged_dt2

,production,full,23k,5k
Italy,0.199843,0.157650,0.253724,0.253711
France,0.170789,0.165685,0.216861,0.216797
Spain,0.154299,0.061265,0.195912,0.195898
US,0.093836,0.505788,0.119138,0.119141
Australia,0.051040,0.021382,0.064796,0.064844
China,0.044759,0.000009,0.000000,0.000000
South Africa,0.041225,0.012052,0.052353,0.052344
Chile,0.039654,0.041160,0.050364,0.050391
Argentina,0.036906,0.035009,0.046851,0.046875


In [42]:

merged_dt2['Country'] = merged_dt2.index
merged_dt2

,production,full,23k,5k,Country
Italy,0.199843,0.157650,0.253724,0.253711,Italy
France,0.170789,0.165685,0.216861,0.216797,France
Spain,0.154299,0.061265,0.195912,0.195898,Spain
US,0.093836,0.505788,0.119138,0.119141,US
Australia,0.051040,0.021382,0.064796,0.064844,Australia
China,0.044759,0.000009,0.000000,0.000000,China
South Africa,0.041225,0.012052,0.052353,0.052344,South Africa
Chile,0.039654,0.041160,0.050364,0.050391,Chile
Argentina,0.036906,0.035009,0.046851,0.046875,Argentina


In [43]:
merged_dt2.to_json("../Resources_cleaned/datasets_vs_production.json",orient='records')

In [140]:
## Lets try to extract also the distributions in price and score of the different datasets

full_distrib = []
for dd in datasets:
    df = pd.read_json(dd['filepath'])
    print(min(df['price']))
    print(max(df['price']))
    nb_bins = max(df['price']) - min(df['price'])
    distrib_price = (df['price']
     .pipe(lambda s: pd.Series(np.histogram(s, range=(4, 300), bins=296)))
     .pipe(lambda s: pd.Series(s[0], index=s[1][:-1]))
     .pipe(lambda s: s/len(df))
                    )
    # There are a lot of gaps in the price data --> we remove the 0 values
#     distrib_price = distrib_price[distrib_price!=0]

    distrib_points = (
        df["points"]
     .pipe(lambda s: pd.Series(np.histogram(s, range=(80,101), bins=21)))
     .pipe(lambda s: pd.Series(s[0], index=s[1][:-1]))
     .pipe(lambda s: s/len(df))
    )


    full_distrib.append({"dataset": dd['name'],
                    "prices":  distrib_price.index.tolist(),
                    "prices_dist": distrib_price.tolist(),
                    "points": distrib_points.index.tolist(),
                    "points_dist": distrib_points.tolist()})
# price = distrib_price.index.tolist()
# price_dist = distrib_price.tolist()
# points = distrib_points.index.tolist()
# points_dist = distrib_points.tolist()

# len(df)
# [0:len(distrib[0])]
# distrib[0]
# distrib[1]
# df_dist = distrib.to_frame().reset_index()
# df_dist = df_dist.rename(columns={ df_dist.columns[0]: "price",
#                                     df_dist.columns[1]: "price_dist"})
# df_dist = df_dist.loc[~(df_dist["price_dist"]==0)]



4
3300
4
3300
4
850


In [141]:
# dist_file = 
# df_dist.to_json(orient='split')
json.dumps(full_distrib)
with open('data.json', 'w') as outfile:
    json.dump(full_distrib, outfile)


In [ ]:
# Calculating p-values
df_full = pd.read_json(datasets[0]['filepath'])
df_23k = pd.read_json(datasets[1]['filepath'])
df_5k = pd.read_json(datasets[2]['filepath'])


In [11]:

df_1k = pd.read_json(r"../Resources_cleaned/winemag_1000.json")

In [4]:
print(stats.ttest_ind(df_23k["price"], df_5k["price"] ))

Ttest_indResult(statistic=-0.2073928543933279, pvalue=0.835704531735038)


In [5]:
print(stats.ttest_ind(df_full["price"], df_23k["price"] ))

Ttest_indResult(statistic=5.454993029251544, pvalue=4.9063139509970316e-08)


In [6]:
print(stats.ttest_ind(df_full["price"], df_5k["price"] ))

Ttest_indResult(statistic=2.603820952833613, pvalue=0.009220302287140516)


In [12]:
print(stats.ttest_ind(df_1k["price"], df_23k["price"] ))

Ttest_indResult(statistic=-1.0926521057529193, pvalue=0.27455728846187133)


In [13]:
print(stats.ttest_ind(df_full["points"], df_23k["points"] ))

Ttest_indResult(statistic=11.801812545569016, pvalue=3.9661028243475246e-32)


In [14]:
print(stats.ttest_ind(df_1k["points"], df_23k["points"] ))

Ttest_indResult(statistic=-0.7854047576036615, pvalue=0.4322238652494438)


In [15]:
print(stats.ttest_ind(df_5k["points"], df_23k["points"] ))

Ttest_indResult(statistic=1.3880451125069082, pvalue=0.1651340658482057)
